In [1]:
!nvidia-smi

Mon Nov 29 14:35:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 48%   60C    P0    37W / 260W |      0MiB / 11016MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import matplotlib.pyplot as plt
import numpy as np
import math

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
batch_size = 2500

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_shape=(3,32,32)):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, 1, bias=False)
        self.conv2 = nn.Conv2d(64, 128, 3, 1, bias=False)
        self.conv3 = nn.Conv2d(128, 256, 3, 1, bias=False)
        self.conv4 = nn.Conv2d(256, 512, 3, 1, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        n_sizes = self._get_conv_output(input_shape)
        self.fc1 = nn.Linear(n_sizes, 512, bias=False)
        self.fc2 = nn.Linear(512, 128, bias=False)
        self.fc3 = nn.Linear(128, 10, bias=False)

    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x
    
    def _forward_features(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        return x

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size




In [16]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 77 %


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
model.load_state_dict(torch.load('cifar10.pth'))

/home/doan.bao.linh/miniconda3/envs/linhdb_env/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448233824/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


<All keys matched successfully>

In [9]:
module_list = [module for module in model.modules() if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear)]

module_shape = [m.weight.shape for m in module_list]

original_weights = [m.weight for m in module_list]

In [10]:
import torch.autograd as autograd
from torch.optim.lr_scheduler import CosineAnnealingLR



class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # Get the supermask by sorting the scores and using the top k%
        out = scores.clone()
        _, idx = scores.flatten().sort()
        j = int((1 - k) * scores.numel())

        # flat_out and out access the same memory.
        flat_out = out.flatten()
        flat_out[idx[:j]] = 0
        flat_out[idx[j:]] = 1

        return out

    @staticmethod
    def backward(ctx, g):
        # send the gradient g straight-through on the backward pass.
        return g, None
    
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # initialize the scores
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        # NOTE: initialize the weights like this.
        nn.init.kaiming_normal_(self.weight, mode="fan_in", nonlinearity="relu")

        # NOTE: turn the gradient on the weights off
        self.weight.requires_grad = False
        self.scores.requires_grad = False

    def forward(self, x):
        subnet = GetSubnet.apply(self.scores.abs(), sparsity)
        w = self.weight * subnet
        x = F.conv2d(
            x, w, self.bias, self.stride, self.padding, self.dilation, self.groups
        )
        
        return x
    
class SupermaskLinear(nn.Linear):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # initialize the scores
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        # NOTE: initialize the weights like this.
        nn.init.kaiming_normal_(self.weight, mode="fan_in", nonlinearity="relu")

        # NOTE: turn the gradient on the weights off
        self.weight.requires_grad = False
        self.scores.requires_grad = False

    def forward(self, x):
        subnet = GetSubnet.apply(self.scores.abs(), sparsity)
        w = self.weight * subnet
        
        return F.linear(x, w, self.bias)

In [12]:
class PruneNet(nn.Module):
    def __init__(self, input_shape=(3,32,32)):
        super(PruneNet, self).__init__()
        self.conv1 = SupermaskConv(3, 64, 3, 1, bias=False)
        self.conv2 = SupermaskConv(64, 128, 3, 1, bias=False)
        self.conv3 = SupermaskConv(128, 256, 3, 1, bias=False)
        self.conv4 = SupermaskConv(256, 512, 3, 1, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        n_sizes = self._get_conv_output(input_shape)
        self.fc1 = SupermaskLinear(n_sizes, 512, bias=False)
        self.fc2 = SupermaskLinear(512, 128, bias=False)
        self.fc3 = SupermaskLinear(128, 10, bias=False)

    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x
    
    def _forward_features(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        return x

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
        return output

In [13]:
torch.manual_seed(seed)
seed = 2206
sparsity = 0.5

In [15]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=2500,
                                         shuffle=False, num_workers=4)

for data, target in testloader:
    test_data, test_target = data.to(device), target.to(device)

Files already downloaded and verified


In [17]:
ga_model = PruneNet().to(device)
scores = []

ga_module_list = [module for module in ga_model.modules() if isinstance(module, SupermaskConv) or isinstance(module, SupermaskLinear)]

# Update weight of ga model with original trained model 

for i, weight in enumerate(original_weights):
    ga_module_list[i].weight = weight

In [20]:
class Agent:
    def __init__(self, params):
        self.params = params
        self.fitness = 0
        
    def set_fitness(self, fitness):
        self.fitness = fitness

In [21]:
# Init population

def init_pop(pop_size=100):
    population = []
    for _ in range(pop_size):
        params = []
        for shape in module_shape:
            scores = nn.Parameter(torch.Tensor(shape))
            nn.init.kaiming_uniform_(scores, a=math.sqrt(5))
            params.append(scores)
        agent = Agent(params=params)
        population.append(agent)
    return population

In [22]:
def change_scores(module_list, agent):
    for i, m_scores in enumerate(agent.params):
        module_list[i].scores = m_scores

In [23]:
def mutation(agent, mut_rate=0.1):
    params = []
    for param in agent.params:
        out = param.clone()
        # flat_out and out share the same memory
        flat_out = out.flatten().to(device)
        # Get index mutation 
        indexes = np.where(np.random.uniform(low=0, high=1, size=(len(flat_out))) < mut_rate)
        replace_values = np.random.uniform(low=-1, high=1, size=(len(flat_out)))[indexes]
        # Mutation
        flat_out.index_copy_(0, torch.LongTensor(indexes[0]).to(device), torch.FloatTensor(replace_values).to(device))
        params.append(nn.Parameter(out))
    return Agent(params=params)

In [24]:
def recombine_agent(agent_1, agent_2):
    params_1 = []
    params_2 = []
    for i, param in enumerate(agent_1.params):
        param_1 = param.clone()
        param_2 = agent_2.params[i].clone()
        # Flatten 
        flat_1 = param_1.flatten().to(device)
        flat_2 = param_2.flatten().to(device)
        # Define children
        child_1 = torch.zeros(len(flat_1))
        child_2 = torch.zeros(len(flat_1))
        # Select cross point
        cross_pt = random.randint(0, len(flat_1))
        # Swap
        child_1[cross_pt:len(flat_1)] = flat_1[cross_pt:len(flat_1)]
        child_1[0:cross_pt] = flat_2[0:cross_pt]
        child_2[cross_pt:len(flat_1)] = flat_2[cross_pt:len(flat_1)]
        child_2[0:cross_pt] = flat_1[0:cross_pt]
        # Append to params 
        params_1.append(nn.Parameter(child_1.reshape(module_shape[i])))
        params_2.append(nn.Parameter(child_2.reshape(module_shape[i])))

    return Agent(params_1), Agent(params_2)

In [25]:
from tqdm import tqdm 

def evaluate_population(pop, data_loader):
    avg_fit = 0
    best_fit = 0
    for agent in tqdm(pop):
        change_scores(ga_module_list, agent)
        fit = test(ga_model.to(device), device, data_loader)
        agent.fitness = fit
        if fit > best_fit: 
            best_fit = fit
        avg_fit += fit
    avg_fit /= len(pop)
    
    return pop, avg_fit, best_fit

In [26]:
def next_generation(pop, size=100, mut_rate=0.01):
    new_pop = []
    while len(new_pop) < size:
        parents = random.choices(pop, k=2, weights=[x.fitness for x in pop])
        offspring_ = recombine_agent(parents[0],parents[1])
        offspring = [mutation(offspring_[0], mut_rate=mut_rate), mutation(offspring_[1], mut_rate=mut_rate)]
        new_pop.extend(offspring) #add offspring to next generation
    return new_pop

In [27]:
def test(model, device, data_loader):
    model.eval()
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    for images, labels in data_loader:
        with torch.no_grad():
            images = images.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total

In [33]:
batch_size = 2500

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [34]:
test(ga_model, device, testloader), test(ga_model, device, trainloader)

(0.512, 0.60658)

In [36]:
import logging
logging.basicConfig(filename=f'cifar10_check_generalization_final_2.log',
                            filemode='a',
                            format='%(asctime)s, %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.INFO)

In [37]:

num_generations = 10000
population_size = 30

pop = init_pop(population_size)

mutation_rate = 0.2 # 0.1% mutation rate

pop_fit = []

pop = init_pop(population_size) # initial population

for gen in range(num_generations):
    # trainning
    pop, avg_fit_test, best_fit = evaluate_population(pop, testloader)
    pop, avg_fit_train, best_fit = evaluate_population(pop, trainloader)
    if avg_fit_test > 0.5:
        population_size = 60
        mutation_rate = 0.15
    if avg_fit_test > 0.6:
        population_size = 100
        mutation_rate = 0.1
    if avg_fit_test > 0.7:
        population_size = 200
        mutation_rate = 0.05
    if avg_fit_test > 0.8:
        population_size = 300
        mutation_rate = 0.01
    avg_fit_full = (avg_fit_train * 6 + avg_fit_test) / 7
    
    message = 'Generation {} with pop_fit: | on_test {} | on_train {} | on_full {}'.format(gen, avg_fit_test, avg_fit_train, avg_fit_full)
    print(message)
    logging.info(message)
    pop_fit.append(avg_fit_test) # record population average fitness
    new_pop = next_generation(pop, size=population_size, mut_rate=mutation_rate)
    pop = new_pop

100%|██████████| 30/30 [02:15<00:00,  4.52s/it]


Generation 0 with pop_fit: | on_test 0.4905166666666666 | on_train 0.5722733333333334 | on_full 0.5605938095238095


100%|██████████| 30/30 [02:16<00:00,  4.56s/it]


Generation 1 with pop_fit: | on_test 0.49876333333333334 | on_train 0.5809933333333334 | on_full 0.5692461904761905


100%|██████████| 30/30 [02:16<00:00,  4.56s/it]


...

Generation 167 with pop_fit: | on_test 0.6041969999999999 | on_train 0.7299968000000001 | on_full 0.7120254


100%|██████████| 100/100 [07:37<00:00,  4.58s/it]


Generation 168 with pop_fit: | on_test 0.606093 | on_train 0.7322253999999999 | on_full 0.7142064857142856


  0%|          | 0/100 [00:04<?, ?it/s]


KeyboardInterrupt: 